<a href="https://colab.research.google.com/github/kavivarshini25/Kavivarshini_INFO5731_FALL2025/blob/main/Kandhasami_Kavivarshini_Assignment_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Monday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (25 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]


(3) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(4) Collect all the information of the 904 narrators in the Densho Digital Repository.

(5)**Collect a total of 10000 reviews** of the top 100 most popular software from G2 and Capterra.


In [2]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import csv
from google.colab import files
page_url = "https://ddr.densho.org/narrators/?page={}"

def extract_narr_info():
    narrs_all = []

    for page_num in range(1, 42):
        wblink_1 = Request(page_url.format(page_num), headers={'User-Agent': 'Mozilla/5.0'})
        wburl_1 = urlopen(wblink_1)
        wbdata_1 = wburl_1.read()
        wbdata_1_soup = BeautifulSoup(wbdata_1, 'html.parser')

        for narr_link in wbdata_1_soup.find_all('h4'):
            wblink_2 = Request(narr_link.a.get('href'), headers={'User-Agent': 'Mozilla/5.0'})
            wburl_2 = urlopen(wblink_2)
            wbdata_2 = wburl_2.read()
            wbdata_2_soup = BeautifulSoup(wbdata_2, 'html.parser')

            narr = wbdata_2_soup.find("div", attrs={'class': 'col-sm-8 col-md-8'})
            narr_name = narr.h1.text.strip().replace('"', "")
            narr_bio = narr.p.text.strip() if narr.p else ""

            interview_titles = []
            dates_locations = []
            densho_ids = []

            for interview in wbdata_2_soup.find_all("div", attrs={'class': 'media'}):
                interview_title = interview.find("b", attrs={'class': 'media-heading'}).text.strip()
                interview_details = interview.find("div", attrs={'class': 'source muted'})
                date_location = ""

                if interview_details:
                    details_text = interview_details.get_text("\n").strip().split("\n")
                    if len(details_text) >= 2:
                        date = details_text[0].strip()
                        location = details_text[1].strip()
                        date_location = f"{date}, {location}"

                interview_link = interview.find("a", href=True)
                densho_id = interview_link['href'].split('/')[-2] if interview_link else ""

                interview_titles.append(interview_title)
                dates_locations.append(date_location)
                densho_ids.append(densho_id)

            narrs_all.append({
                'narr Name': narr_name,
                'Bio': narr_bio,
                'Interview Title': ", ".join(interview_titles),
                'Date & Location': ", ".join(dates_locations),
                'Densho ID': ", ".join(densho_ids)
            })

    csv_file = "List_of_narrs_collected.csv"
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=[
            'narr Name', 'Bio', 'Interview Title', 'Date & Location', 'Densho ID'
        ])
        writer.writeheader()
        writer.writerows(narrs_all)

    print(f"All narrs' data saved to '{csv_file}'.")
    files.download(csv_file)

extract_narr_info()




All narrs' data saved to 'List_of_narrs_collected.csv'.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Question 2 (15 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [3]:
# Write code for each of the sub parts with proper comments.
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from google.colab import files

# Download NLTK resources (only once)
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Load your CSV
my_file = pd.read_csv("List_of_narrs_collected.csv")

# Initialize stemmer and lemmatizer
stmr = PorterStemmer()
lemtzr = WordNetLemmatizer()
stp_wrds = set(stopwords.words('english'))

def final_clean_txt(text, show_steps=False):
    if pd.isna(text):
        return ""

    original_text = text

    # 1. Remove noise: special characters and punctuations
    text_no_noise = re.sub(r"[^\w\s]", " ", original_text)

    # 2. Remove numbers
    text_no_numbers = re.sub(r"\d+", "", text_no_noise)

    # 3. Lowercase all text
    text_lower = text_no_numbers.lower()

    # 4. Tokenize and remove stopwords
    words = text_lower.split()
    words_no_stop = [word for word in words if word not in stp_wrds]

    # 5. Stemming
    words_stemmed = [stmr.stem(word) for word in words_no_stop]

    # 6. Lemmatization
    words_lemmatized = [lemtzr.lemmatize(word) for word in words_stemmed]

    final_cleaned_txt = " ".join(words_lemmatized)

    # Show intermediate results for one sample
    if show_steps:
        print("Original Text:\n", original_text, "\n")
        print("Step 1 - No Noise:\n", text_no_noise, "\n")
        print("Step 2 - No Numbers:\n", text_no_numbers, "\n")
        print("Step 3 - Lowercase:\n", text_lower, "\n")
        print("Step 4 - Stopwords Removed:\n", " ".join(words_no_stop), "\n")
        print("Step 5 - Stemming:\n", " ".join(words_stemmed), "\n")
        print("Step 6 - Lemmatization (Final Cleaned):\n", final_cleaned_txt, "\n")

    return final_cleaned_txt

# Apply cleaning to the Bio column
my_file['Cleaned Bio'] = my_file['Bio'].apply(final_clean_txt)

# Show cleaning steps for the first Bio
print("=== Cleaning Steps for First Bio ===")
_ = final_clean_txt(my_file['Bio'].iloc[0], show_steps=True)

# Show first 5 bios (original vs cleaned)
print("\n=== Preview of First 5 Bios (Original vs Cleaned) ===")
print(my_file[['Bio', 'Cleaned Bio']].head(5))

# Save to new CSV
fout_fl = "Final_List_of_narrs_cleaned.csv"
my_file.to_csv(fout_fl, index=False)
print("\n Cleaned data is saved to 'Final_List_of_narrs_cleaned.csv'.")

# Download file directly in Colab
files.download(fout_fl)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


=== Cleaning Steps for First Bio ===
Original Text:
 Nisei female. Born May 9, 1927, in Selleck, Washington. Spent much of childhood in Beaverton, Oregon, where father owned a farm. Influenced at an early age by parents' conversion to Christianity. During World War II, removed to the Portland Assembly Center, Oregon, and the Minidoka concentration camp, Idaho. After the war, worked to establish a successful volunteer program to feed the homeless in Seattle, Washington. 

Step 1 - No Noise:
 Nisei female  Born May 9  1927  in Selleck  Washington  Spent much of childhood in Beaverton  Oregon  where father owned a farm  Influenced at an early age by parents  conversion to Christianity  During World War II  removed to the Portland Assembly Center  Oregon  and the Minidoka concentration camp  Idaho  After the war  worked to establish a successful volunteer program to feed the homeless in Seattle  Washington  

Step 2 - No Numbers:
 Nisei female  Born May     in Selleck  Washington  Spent mu

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Question 3 (15 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [5]:
# Your code here
# 1. Install and import required libraries
!pip install spacy
import pandas as pd
import spacy
from collections import Counter

# 2. Load spaCy English model
!python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

# 3. Load cleaned CSV file
my_file = "Final_List_of_narrs_cleaned.csv"
fp = pd.read_csv(my_file, dtype=str).fillna('')

# 4. ===============================
# PART 1: POS TAGGING
# ===============================
def pst_anal(text):
    """Count Nouns, Verbs, Adjectives, Adverbs in a text"""
    if not isinstance(text, str) or text.strip() == "":
        return {'N': 0, 'V': 0, 'Adj': 0, 'Adv': 0}

    recrd = nlp(text)
    po_cnts = {'N': 0, 'V': 0, 'Adj': 0, 'Adv': 0}
    for token in recrd:
        if token.pos_ == "NOUN":
            po_cnts['N'] += 1
        elif token.pos_ == "VERB":
            po_cnts['V'] += 1
        elif token.pos_ == "ADJ":
            po_cnts['Adj'] += 1
        elif token.pos_ == "ADV":
            po_cnts['Adv'] += 1
    return po_cnts

# Apply POS analysis to the 'Cleaned Bio' column
fp['po_cnts'] = fp['Cleaned Bio'].apply(pst_anal)
pos_fp = pd.DataFrame(fp['po_cnts'].tolist())

# Save individual POS counts
pos_fp.to_csv("/content/pst_anal_results.csv", index=False)

# Save total counts
tt_cnts = pd.DataFrame([pos_fp.sum()])
tt_cnts.to_csv("/content/Total_po_cnts.csv", index=False)

print("POS Analysis Completed!")
print("Individual counts saved to pst_anal_results.csv")
print("Total counts saved to Total_po_cnts.csv\n")

# 5. ===============================
# PART 2: Dependency Parsing
# ===============================
def parse_dependencies(text):
    """Return dependency parse: (word, relation, head)"""
    recrd = nlp(text)
    return [(token.text, token.dep_, token.head.text) for token in recrd]

# Use first narrator as example
if "Cleaned Bio" in fp.columns and not fp.empty:
    example_text = fp.loc[0, 'Cleaned Bio']
    dep_tree = parse_dependencies(example_text)

    print("=== Dependency Parsing for First Narrator ===")
    for word, relation, head in dep_tree:
        print(f"{word:15} -> {relation:10} (head: {head})")
else:
    print("No valid 'Cleaned Bio' column found.")

# 6. ===============================
# PART 3: Named Entity Recognition (NER)
# ===============================
ctp = ["Narrator Name", "Cleaned Bio", "Interview Title", "Date & Location"]
ec = Counter()
unique_entities = set()

for column in ctp:
    if column in fp.columns:
        for text in fp[column].dropna():
            recrd = nlp(text)
            for ent in recrd.ents:
                unique_entities.add((ent.text, ent.label_))

# Count unique entities per label
for _, label in unique_entities:
    ec[label] += 1

print("\n=== Named Entity Recognition Counts ===")
for entity, count in ec.items():
    print(f"{entity}: {count}")



  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
POS Analysis Completed!
Individual counts saved to pst_anal_results.csv
Total counts saved to Total_po_cnts.csv

=== Dependency Parsing for First Narrator ===
nisei           -> compound   (head: femal)
femal           -> nsubj      (head: spent)
born            -> acl        (head: femal)
may             -> aux        (head: spent)
selleck         -> compound   (head: washington)
washington      -> nsubj      (head: spent)
spent           -> ROOT       (head: spent)
much            -> amod       (head: beaverton)


# **Following Questions must answer using AI assitance**

#Question 4 (20 points).

Q4. (PART-1)
Web scraping data from the GitHub Marketplace to gather details about popular actions. Using Python, the process begins by sending HTTP requests to multiple pages of the marketplace (1000 products), handling pagination through dynamic page numbers. The key details extracted include the product name, a short description, and the URL.

 The extracted data is stored in a structured CSV format with columns for product name, description, URL, and page number. A time delay is introduced between requests to avoid server overload. ChatGPT can assist by helping with the parsing of HTML, error handling, and generating reports based on the data collected.

 The goal is to complete the scraping within a specified time limit, ensuring that the process is efficient and adheres to GitHub’s usage guidelines.

(PART -2)

1.   **Preprocess Data**: Clean the text by tokenizing, removing stopwords, and converting to lowercase.

2. Perform **Data Quality** operations.


Preprocessing:
Preprocessing involves cleaning the text by removing noise such as special characters, HTML tags, and unnecessary whitespace. It also includes tasks like tokenization, stopword removal, and lemmatization to standardize the text for analysis.

Data Quality:
Data quality checks ensure completeness, consistency, and accuracy by verifying that all required columns are filled and formatted correctly. Additionally, it involves identifying and removing duplicates, handling missing values, and ensuring the data reflects the true content accurately.


Github MarketPlace page:
https://github.com/marketplace?type=actions

Prompt:Give me a code to scrape datafrom the GitHub Marketplace to get informations like product name, description, URL, and page number and format it in a csv file.Give me python code to use in Google collab.

In [7]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

from bs4 import BeautifulSoup
import requests
import csv
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from requests.adapters import HTTPAdapter

# Session setup with larger connection pool

session = requests.Session()
adapter = HTTPAdapter(pool_connections=50, pool_maxsize=50)  # increase pool size
session.mount('https://', adapter)
session.mount('http://', adapter)
session.headers.update({'User-Agent': 'Mozilla/5.0'})

# --------------------------
# Exponential backoff
# --------------------------
def sleep_with_backoff(attempt):
    time.sleep(min(2 ** attempt + random.uniform(0, 1), 10))

# --------------------------
# Extract actions from a page
# --------------------------
def extract_actn_lnk(page_number):
    wblk = f"https://github.com/marketplace?type=actions&page={page_number}"
    actn_lnk = []
    print(f"Processing Page {page_number}")

    for attempt in range(5):
        try:
            response = session.get(wblk, timeout=10)
            if response.status_code != 200:
                raise Exception(f"Status code: {response.status_code}")

            soup = BeautifulSoup(response.content, 'html.parser')
            actions = soup.find_all("div", attrs={'data-testid': 'non-featured-item'})

            for action in actions:
                ln_tg = action.find("a", href=True)
                name = ln_tg.text.strip() if ln_tg else "N/A"
                link = f"https://github.com{ln_tg['href']}" if ln_tg else "N/A"
                actn_lnk.append({
                    "Page Number": page_number,
                    "Action Name": name,
                    "Link": link,
                    "Description": "N/A"
                })
            break  # success
        except Exception as e:
            print(f"Error on page {page_number}: {e}")
            sleep_with_backoff(attempt)

    print(f"✅ Extracted {len(actn_lnk)} actions from page {page_number}")
    return actn_lnk

# --------------------------
# Fetch description for an action
# --------------------------
def fetch_description(action):
    wblk = action['Link']
    for attempt in range(5):
        try:
            response = session.get(wblk, timeout=10)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'lxml')
                div = soup.find('div', attrs={'data-testid': 'about'})
                if div:
                    span = div.find('span')
                    if span:
                        action['Description'] = span.get_text().strip()
            break
        except Exception as e:
            print(f"Error fetching {wblk}: {e}")
            sleep_with_backoff(attempt)
    return action

# --------------------------
# Save CSV
# --------------------------
def save_to_csv(data, filename):
    with open(filename, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['Page Number', 'Action Name', 'Link', 'Description'])
        writer.writeheader()
        writer.writerows(data)

# --------------------------
# Main scraping
# --------------------------
if __name__ == "__main__":
    all_actions = []
    max_pages = 65  # adjust if needed

    # Scrape all pages
    for page_number in range(1, max_pages + 1):
        actions = extract_actn_lnk(page_number)
        all_actions.extend(actions)
        if page_number % 5 == 0:
            print(f"Progress: {len(all_actions)} actions collected after {page_number} pages")
        time.sleep(random.uniform(1, 2))  # small delay between pages

    print(f"Total actions extracted before deduplication: {len(all_actions)}")

    # Remove duplicates by Link
    unique_actions = {a['Link']: a for a in all_actions if a['Link'] != "N/A"}
    unique_actions = list(unique_actions.values())
    print(f"Total unique actions to fetch descriptions: {len(unique_actions)}")

    # --------------------------
    # Concurrent description fetching
    # --------------------------
    print("Fetching descriptions concurrently...")
    results = []
    with ThreadPoolExecutor(max_workers=20) as executor:
        futre = [executor.submit(fetch_description, a) for a in unique_actions]
        for f in tqdm(as_completed(futre), total=len(futre)):
            results.append(f.result())

    # Save final CSV
    save_to_csv(results, "github_actions_final_data.csv")
    print(f"✅ Data extraction completed. Total actions saved: {len(results)}")


Processing Page 1
✅ Extracted 20 actions from page 1
Processing Page 2
✅ Extracted 20 actions from page 2
Processing Page 3
✅ Extracted 20 actions from page 3
Processing Page 4
✅ Extracted 0 actions from page 4
Processing Page 5
✅ Extracted 20 actions from page 5
Progress: 80 actions collected after 5 pages
Processing Page 6
✅ Extracted 20 actions from page 6
Processing Page 7
✅ Extracted 0 actions from page 7
Processing Page 8
✅ Extracted 20 actions from page 8
Processing Page 9
✅ Extracted 20 actions from page 9
Processing Page 10
✅ Extracted 20 actions from page 10
Progress: 160 actions collected after 10 pages
Processing Page 11
✅ Extracted 0 actions from page 11
Processing Page 12
✅ Extracted 20 actions from page 12
Processing Page 13
✅ Extracted 20 actions from page 13
Processing Page 14
✅ Extracted 0 actions from page 14
Processing Page 15
✅ Extracted 20 actions from page 15
Progress: 220 actions collected after 15 pages
Processing Page 16
✅ Extracted 20 actions from page 16
Pro

100%|██████████| 1174/1174 [03:42<00:00,  5.28it/s]

✅ Data extraction completed. Total actions saved: 1174


Prompt: Give me the python code to clean the extraced file by utilising the necessary libraries for it

In [11]:
# --------------------------
# Import Libraries
# --------------------------
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# --------------------------
# Download NLTK resources (first time)
# --------------------------
nltk.download("punkt")
nltk.download("punkt_tab")   # fixes LookupError in newer NLTK versions
nltk.download("stopwords")
nltk.download("wordnet")

# --------------------------
# Load Data
# --------------------------
my_file = "github_actions_final_data.csv"  # Update path if needed
datafile = pd.read_csv(my_file)
print("📊 Original dataset shape:", datafile.shape)

# --------------------------
# Text Preprocessing Function
# --------------------------
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    if pd.isna(text):
        return "no description available"

    # Remove HTML tags
    text = re.sub(r"<.*?>", " ", str(text))
    # Keep only letters and spaces
    text = re.sub(r"[^a-zA-Z\s]", " ", text)
    # Lowercase and strip
    text = text.lower().strip()
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

# --------------------------
# Data Quality Checks
# --------------------------
def data_quality_checks(data):
    # Remove duplicates by unique identifier (Link)
    before = data.shape[0]
    data = data.drop_duplicates(subset=["Link"])
    after = data.shape[0]
    print(f"🔍 Removed {before - after} duplicate rows")

    # Fill missing values
    data["Action Name"] = data["Action Name"].fillna("Unknown")
    data["Description"] = data["Description"].fillna("No description available")

    return data

# --------------------------
# Apply Preprocessing (create new columns)
# --------------------------
datafile["Clean_Action_Name"] = datafile["Action Name"].apply(preprocess_text)
datafile["Clean_Description"] = datafile["Description"].apply(preprocess_text)

# --------------------------
# Perform Data Quality
# --------------------------
final_data = data_quality_checks(datafile)
print("📊 Dataset shape after cleaning:", final_data.shape)

# --------------------------
# Save Processed Data
# --------------------------
output_file = "github_actions_processed_data.csv"
final_data.to_csv(output_file, index=False, encoding="utf-8")
print(f"✅ Data preprocessing & quality checks completed. File saved: {output_file}")

# --------------------------
# Mini Data Quality Report
# --------------------------
print("\n--- Data Quality Report ---")
print("Null values per column:\n", final_data.isnull().sum())
print("\nAverage description length (words):",
      final_data["Clean_Description"].apply(lambda x: len(x.split())).mean())
print("Average action name length (words):",
      final_data["Clean_Action_Name"].apply(lambda x: len(x.split())).mean())

# Optional: Top 20 most common words in descriptions
from collections import Counter
all_words = " ".join(final_data["Clean_Description"]).split()
most_common_words = Counter(all_words).most_common(20)
print("\nTop 20 most common words in descriptions:")
for word, count in most_common_words:
    print(f"{word}: {count}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


📊 Original dataset shape: (1174, 4)
🔍 Removed 0 duplicate rows
📊 Dataset shape after cleaning: (1174, 6)
✅ Data preprocessing & quality checks completed. File saved: github_actions_processed_data.csv

--- Data Quality Report ---
Null values per column:
 Page Number          0
Action Name          0
Link                 0
Description          0
Clean_Action_Name    0
Clean_Description    0
dtype: int64

Average description length (words): 6.904599659284497
Average action name length (words): 2.82793867120954

Top 20 most common words in descriptions:
github: 401
action: 378
run: 136
request: 105
file: 101
pull: 101
code: 94
build: 81
using: 75
workflow: 68
release: 59
repository: 57
pr: 54
check: 54
deploy: 52
project: 51
automatically: 51
version: 50
issue: 50
setup: 48


#Question 5 (20 points)

PART 1:
Web Scrape  tweets from Twitter using the Tweepy API, specifically targeting hashtags related to subtopics (machine learning or artificial intelligence.)
The extracted data includes the tweet ID, username, and text.

Part 2:
Perform data cleaning procedures

A final data quality check ensures the completeness and consistency of the dataset. The cleaned data is then saved into a CSV file for further analysis.


**Note**

1.   Follow tutorials provided in canvas to obtain api keys. Use ChatGPT to get the code. Make sure the file is downloaded and saved.
2.   Make sure you divide GPT code as shown in tutorials, dont make multiple requestes.


Prompt:Give me the code to web scrape from Twitter using the Tweepy API using python. Tell me the step by step procedure to scrap specific hashtags related to subtopics like machine learning or artificial intelligence.The extracted data includes the tweet ID, username, and text.

api key - XRiewqo35I94XlnBkRMpsxUbR
api-key secret-9g8NsUK8JgABAk0k92lZRaaGfDTnUEwaG2knf1WpOwSRTKzcQJ
bearer token - AAAAAAAAAAAAAAAAAAAAAJT54QEAAAAABKCgw%2BATC%2FCgpGXYDVf6CqrUYf8%3Dd9OVqYgYOUW88x21AHVoiAttD9NSvLW9nWa5e4LgPTtXKFuMFz
access token -1972857394087485440-7GLF55yrz883lvkxDyYZ1bN0Y8AniU
access token secret -ChrWZwsFmLx13JomUBrCtDSGakt5o5rIU8JiGTsLu46O7
ChrWZwsFmLx13JomUBrCtDSGakt5o5rIU8JiGTsLu46O7

In [35]:
import tweepy
import pandas as pd

In [36]:
# Twitter API credentials
api_key = "XRiewqo35I94XlnBkRMpsxUbR"
api_key_secret = "9g8NsUK8JgABAk0k92lZRaaGfDTnUEwaG2knf1WpOwSRTKzcQJ "
access_token = "1972857394087485440-7GLF55yrz883lvkxDyYZ1bN0Y8AniU"
access_token_secret = "ChrWZwsFmLx13JomUBrCtDSGakt5o5rIU8JiGTsLu46O7"

# Authenticate with Twitter API
auth = tweepy.OAuth1UserHandler(api_key, api_key_secret, access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [37]:
query = "(#MachineLearning OR #ArtificialIntelligence OR #AI OR #ML) -is:retweet"

# Use Twitter API v2
client = tweepy.Client(bearer_token="AAAAAAAAAAAAAAAAAAAAAJT54QEAAAAABKCgw%2BATC%2FCgpGXYDVf6CqrUYf8%3Dd9OVqYgYOUW88x21AHVoiAttD9NSvLW9nWa5e4LgPTtXKFuMFz")

tweets = client.search_recent_tweets(query=query,
                                     tweet_fields=["created_at", "text", "author_id"],
                                     max_results=100)

# Store the tweets
tweet_data = []

for tweet in tweets.data:
    tweet_data.append({
        'tweet_id': tweet.id,
        'created_at': tweet.created_at,
        'user': tweet.author_id,  # API v2 does not return screen_name directly
        'text': tweet.text
    })

In [38]:
print(len(tweet_data))  # Check how many tweets were stored
print(tweet_data[:12])   # Print first 12 tweets for verification

100
[{'tweet_id': 1972858530324734029, 'created_at': datetime.datetime(2025, 9, 30, 2, 58, 22, tzinfo=datetime.timezone.utc), 'user': 1663161949524226055, 'text': 'Clone your voice in 11 seconds 🎤\n…without the heavy accent 👀\nHere’s the trick:\n✨ Record easy sentences\n✨ Pick Cartesia or Fish Audio\n✨ Add emotion tags for extra flair\n🧐Try It Today! 👉https://t.co/pn0bE6ugOF\n\n🎥 Watch the full tutorial 👇\n\n#AI #VoiceCloning #TTS https://t.co/S7uwZXhlwL'}, {'tweet_id': 1972858525622890755, 'created_at': datetime.datetime(2025, 9, 30, 2, 58, 21, tzinfo=datetime.timezone.utc), 'user': 2913601369, 'text': '🤡不是说Sonnet 4.5很牛逼吗？怎么使用感觉有点降智呢？难不成任务难度很大？一个给出demo和文档的简单任务都完成不了！！！\n\n#独立开发者 #buildinpublic #indiehacker #indiehackers #AI #Apple'}, {'tweet_id': 1972858509265105393, 'created_at': datetime.datetime(2025, 9, 30, 2, 58, 17, tzinfo=datetime.timezone.utc), 'user': 1918103693800685568, 'text': '🎨ブログを公開しました✨\nテーマは『AIクリエイターの仕事とは？』👩\u200d💻\nAIを活用したデザインや動画編集を通して、身につくスキルをご紹介🌿\n未経験から挑戦したい方も大歓迎です！

In [39]:
# Convert to DataFrame
df = pd.DataFrame(tweet_data)

# Save as CSV file
df.to_csv("tweets.csv", index=False, encoding="utf-8")

print("Tweets saved successfully to tweets.csv")

Tweets saved successfully to tweets.csv


In [40]:
df_check = pd.read_csv("tweets.csv")
print(df_check.head())

              tweet_id                 created_at                 user  \
0  1972858530324734029  2025-09-30 02:58:22+00:00  1663161949524226055   
1  1972858525622890755  2025-09-30 02:58:21+00:00           2913601369   
2  1972858509265105393  2025-09-30 02:58:17+00:00  1918103693800685568   
3  1972858498091561255  2025-09-30 02:58:14+00:00  1954780698222723072   
4  1972858495503650943  2025-09-30 02:58:13+00:00           1870696021   

                                                text  
0  Clone your voice in 11 seconds 🎤\n…without the...  
1  🤡不是说Sonnet 4.5很牛逼吗？怎么使用感觉有点降智呢？难不成任务难度很大？一个给出d...  
2  🎨ブログを公開しました✨\nテーマは『AIクリエイターの仕事とは？』👩‍💻\nAIを活用した...  
3  🌘👱🎌🛐\n💵 $128K this weekend ✅ Stock expert @Mul...  
4  Beware of the “Recovery” and “Advance-Fee Reco...  


In [41]:
# 1️⃣ Install missing packages
!pip install emoji unidecode textblob

# 2️⃣ Import libraries
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import emoji
import unidecode

# 3️⃣ Download NLTK resources
nltk.download("stopwords")
nltk.download("wordnet")

# 4️⃣ Load CSV
df = pd.read_csv("/content/tweets.csv")

# 5️⃣ Data cleaning & preprocessing (as in your code)
# Remove duplicates
df.drop_duplicates(subset="tweet_id", keep="first", inplace=True)
df.dropna(inplace=True)

def clean_text(text):
    text = unidecode.unidecode(text)
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"@[A-Za-z0-9_]+", "", text)
    text = re.sub(r"#(\w+)", r"\1", text)
    text = re.sub(r"[^\w\s]", "", text)
    text = emoji.replace_emoji(text, replace="")
    text = re.sub(r"\s+", " ", text).strip()
    return text.lower()

df["text"] = df["text"].apply(clean_text)

# Remove stopwords & lemmatize
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return " ".join(words)

df["text"] = df["text"].apply(preprocess_text)

# Spell correction
def correct_spelling(text):
    return str(TextBlob(text).correct())

df["text"] = df["text"].apply(correct_spelling)

# Convert 'created_at' to datetime
df["created_at"] = pd.to_datetime(df["created_at"])

# Final checks
print("Final dataset shape:", df.shape)
print(df.head())

# Save cleaned CSV
cleaned_filename = "final_cleaned_tweets.csv"
df.to_csv(cleaned_filename, index=False, encoding="utf-8")
print(f"Deep cleaning complete! Cleaned data saved as '{cleaned_filename}'.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 18.3 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Final dataset shape: (100, 4)
              tweet_id                created_at                 user  \
0  1972858530324734029 2025-09-30 02:58:22+00:00  1663161949524226055   
1  1972858525622890755 2025-09-30 02:58:21+00:00           2913601369   
2  1972858509265105393 2025-09-30 02:58:17+00:00  1918103693800685568   
3  1972858498091561255 2025-09-30 02:58:14+00:00  1954780698222723072   
4  1972858495503650943 2025-09-30 02:58:13+00:00           1870696021   

                                                text  
0  alone voice 11 second without heavy accent her...  
1  bu she shut bonnet when nip i zen to she long ...  
2  buroguwogong ai shimashita temahaaikurieitanos...  
3  128k weekend stock expert know he follow hot p...  
4  beware recovery advancefee recovery scar read ...  
Deep cleaning complete! Cleaned data saved as 'final_cleaned_tweets.csv'.


# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

# Write your response below
Fill out survey and provide your valuable feedback.

https://docs.google.com/forms/d/e/1FAIpQLSd_ObuA3iNoL7Az_C-2NOfHodfKCfDzHZtGRfIker6WyZqTtA/viewform?usp=dialog